앞에서는 $x_0$과 $x_1$의 편미분을 변수별로 따로 계산했다. 그런데 이를 동시에 계산하고 싶다면 어떻게 해야 할까? 가령 $x_0=3$, $x_1=4$ 일 때, $(x_0, x_1)$ 양쪽의 편미분을 묶어서 $({\partial{f} \over x_0},{\partial{f} \over x_1})$을 계산해보자. 이 때, $({\partial{f} \over x_0},{\partial{f} \over x_1})$ 처럼 모든 변수의 편미분을 벡터로 정리한 것을 $기울기^{gradient}$라고 한다. 기울기는 예를 들어 다음과 같이 구할 수 있다.

In [57]:
import numpy as np

def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)

    for idx in range(x.size):
        # f(x+h) 계산
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)

        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h) 
        x[idx] = tmp_val # 값 복원

    # print('grad:', grad)

    return grad

def function_2(x):
    return x[0]**2 + x[1]**2

★★★★ 트러블슈팅 ★★★★

> 위의 식에서 2*h를 반드시 괄호로 묶어줘야 한다... 그렇지 않으면 부동소수점이 2로 나누고 h를 곱해주는 걸로 인식해서 결과값이 매우 작게 나온다.

In [58]:
print(numerical_gradient(function_2, np.array([3.0, 4.0])))
print(numerical_gradient(function_2, np.array([0.0, 2.0])))
print(numerical_gradient(function_2, np.array([3.0, 0.0])))

[6. 8.]
[0. 4.]
[6. 0.]


각각의 기울기를 정리하면 아래와 같다. <br>
* (3, 4) -> grad: (6.0, 8.0)
* (0, 2) -> grad: (0.0, 4.0)
* (3, 0) -> grad: (6.0, 0.0)

이러한 기울기들은 무엇을 의미하는 걸까? 그림으로 그려보면 이해가 될 것이다. 다만, 여기에서는 기울기의 결과에 마이너스를 붙인 백터를 그려보겠다. 기울기 그림은 아래처럼 방향을 가진 벡터(화살표)로 그려진다. 이 그림을 보면 기울기는 함수의 가장 낮은 장소를 가리키는 것 같다. 마치 나침반처럼 화살표들은 한 점을 향하고 있다. '가장 낮은 곳'에서 멀어질수록 화살표의 크기도 커짐을 알 수 있다.

<p align="center"><img src="imgs/4-9.png" width=500></p>

위의 그림에서 기울기는 가장 낮은 장소를 가리키지만, 실제로는 반드시 그렇다고도 할 수 없다. 사실 기울기는 각 지점에서 낮아지는 방향을 의미한다. 더 정확히 말하면, 기울기가 가리키는 쪽은 각 장소에서 함수의 출력값을 가장 크게 줄이는 방향인 것이다.

# 4.4.1 경사법(경사하강법)

기계학습 문제 대부분은 학습 단계에서 최적의 매개변수를 찾아낸다. 신경망 역시 최적의 매개변수를 학습 시에 찾아야 한다. 여기서 최적이란, 손실함수가 최소값이 될 때를 의미한다. 그러나 일반적인 문제의 손실함수는 매우 복잡해서 어디가 최솟점인지 찾기가 어렵다. 이럴 때 기울기를 이용하여, 함수의 최소 지점을 찾으려는 것이 경사 하강법이다. 여기서 주의할 점은 각 지점에서 함수의 값을 낮추는 지표가 기울기라는 것이다. 그러나 기울기가 가리키는 곳에 정말 함수의 최소값이 있는지는 보장할 수 없다. 실제로 복잡한 함수에서는 기울기가 가리키는 곳에 최솟값이 없는 경우가 대부분이다.
<br>
기울어진 방향이 꼭 최소값을 가리키는 것은 아니나, 그 방향으로 가야 함수의 값을 줄일 수 있다. 그래서 사실 선택의 여지가 별로 없긴 하다. (이 방법에서는!) 드디어 경사하강법을 소개한다. 경사하강법은 현 위치에서 기울어진 방향만큼 이동하는 것이다. 경사하강법을 수식으로 나타내면 아래와 같다.

$$\huge{x_0 = x_0 - \eta{\partial f \over \partial x_0}}$$
$$\huge{x_1 = x_1 - \eta{\partial f \over \partial x_1}}$$

위의 식에서 $\eta^{에타}$는 생신하는 양을 나타낸다. 이를 learning rate라고 한다. 한 번의 학습으로 얼마만큼을 학습해야 할 지, 즉 매개변수의 값을 얼마나 갱신하는 지를 정하는 것이 학습률이다. 위의 식은 1회에 해당하는 것이고 이를 여러 번 반복하면서 서서히 함수의 값을 줄여나가게 된다. 또 여기에서는 변수가 2개인 경우만 살펴봤지만, 변수가 늘어도 같은 식으로 갱신이 된다. 또한 학습률은 0.01이나 0.001 등 미리 특정 값으로 정해둬야 하는데, 일반적으로 너무 크거나 너무 작으면 '좋은 장소'를 찾아갈 수 없다. 경사 하강법은 다음과 같이 간단하게 구현할 수 있다.

In [61]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x

    for i in range(step_num):
        grad = numerical_gradient(f, x)
        # print('grad:', grad)
        x -= (lr * grad)

    return x

문제. 경사법으로 $f(x_0, x_1) = x_0^2+x_1^2$의 최솟값을 구하여라

In [62]:
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x, 0.1, 100)

array([-6.11110793e-10,  8.14814391e-10])

결과값이 매우 작게 나타나는데, 이는 거의 0에 가까운 값이다. 실제로 진성한 최소값은 (0, 0) 이므로 경사하강법으로 거의 정확한 결과를 얻은 것이나 다름 없다. 경사 하강법을 이용하여 갱신하는 과정을 그려보면 아래 그림과 같이 된다.

<p align="center"><img src="imgs/4-10.png" width=500></p>

다음 코드는 학습률에 따른 결과값을 보여주는데, 학습률이 너무 크면 발산해버리고 학습률이 너무 작으면 원하는 목표지점에 도달하지 못함을 보여주고 있다.

In [63]:
# 학습률이 너무 클 때
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x, 10, 100)

array([-2.58983747e+13, -1.29524862e+12])

In [64]:
# 학습률이 너무 작을 때
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x, 0.001, 100)

array([-2.45570041,  3.27426722])

# 4.4.2 신경망에서의 기울기

신경망 학습에서도 기울기를 구해야 한다. 여기서 말하는 기울기는 가중치 매개변수에 대한 손실 함수의 기울기를 말한다. 예를 들어 형상이 2 x 3, 가중치가 W, 손실함수가 L인 신경망을 생각해보자. 이 경우 경사는 ${\partial L \over \partial W}$로 나타낼 수 있다. 수식으로는 다움과 같다.

$$ W=\left[
\begin{array}{cc}
   w_{11} & w_{12} & w_{13}\\
   w_{21} & w_{22} & w_{23}\\
\end{array}
\right] $$

$$
{\partial L \over \partial W} = {
      \left[
         \begin{array}{cc}
            {\partial L \over \partial w_{11}} & 
            {\partial L \over \partial w_{12}} & 
            {\partial L \over \partial w_{13}} \\
            {\partial L \over \partial w_{21}} &
            {\partial L \over \partial w_{22}} &
            {\partial L \over \partial w_{23}} \\
         \end{array}
   \right]
}
$$

$\partial L \over \partial W$ 의 각 원소는 각각의 원소에 대한 편미분이다. 여기서 중요한 것은 각각의 형상이 W와 동일하다는 점이다. 그럼 간단한 예를 들어 실제로 기울기를 구하는 코드를 구현해보겠다. 먼저 simpleNet class이다.

In [36]:
import sys, os
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class SimpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        # print('x:', x)
        # print('t:', t)
        z = self.predict(x)

        y = softmax(z)
        # print('y:', y)
        loss_rst = cross_entropy_error(y, t)
        # print('loss:', loss_rst)

        return loss_rst

In [37]:
net = SimpleNet()
net.W

array([[ 0.34617015, -1.01858978,  0.98591091],
       [ 1.37906967,  0.26579902,  0.42908762]])

In [39]:
x = np.array([0.6, 0.9])
p = net.predict(x)
p

array([ 1.44886479, -0.37193475,  0.9777254 ])

In [40]:
np.argmax(p)

0

In [41]:
t = np.array([0, 0, 1])
net.loss(x, t)

1.051222177891379

이어서 기울기를 구해보자. 지금가지처럼 gradient 함수를 써서 구하면 된다. 더미 함수 f(W)를 새로 정의해서 계산해보자.

In [42]:
def f(W):
    # x : np.array([0.6, 0.9])
    # t : np.array([0, 0, 1])
    return net.loss(x, t)

dw = numerical_gradient(f, net.W)
print(dw)

[[ 0.33591102  0.05438274 -0.39029375]
 [ 0.50386653  0.0815741  -0.58544063]]


numerical_gradient(f, x)의 인수 f는 함수, x는 함수 f의 인수이다. 그리고 여기에서는 net.W를 인수로 받아 손실함수를 계산하는 새로운 함수 f를 정의하였다. 위에서 나온 값들은 매 번 바뀌기 때문에, 아래에 고정시켜두었다. <br><br>

[[ 0.33591102  0.05438274 -0.39029375] <br>
 [ 0.50386653  0.0815741  -0.58544063]]<br><br>

행렬의 첫 번째 요소를 살펴보면, 0.33 정도가 나왔다. 이는 $w_{11}$를 h만큼 늘리면, 손실함수는 0.33h 만큼 증가한다는 의미이다. 손실함수가 증가한다는 건 안 좋은 것이니, 그만큼 반대의 방향으로 가주어야 한다. 그래서 행렬의 요소에 마이너스 값을 곱해주어, 반대의 방향으로 손실함수가 움직이게 하는 것이다. 참고로 아래와 같이 람다 함수를 사용해도 동일한 결과를 얻을 수 있다.

In [45]:
f = lambda w: net.loss(x, t)
dw = numerical_gradient(f, net.W)
dw

array([[ 0.33591102,  0.05438274, -0.39029375],
       [ 0.50386653,  0.0815741 , -0.58544063]])